# **PCA (II)**
## Las matemáticas detrás del Análisis de Componentes Principales    
 


In [29]:
# Librerías para la preparación de datos
import pandas as pd
import numpy as np
import itertools

# Librerías para la Visualización 
import matplotlib.pyplot as plt

## Obtención y preparación de los datos  
En primer lugar descargamos el dataset de nuestra elección. En esta ocasión usraemos uno creado a partir de datos provinciales proporcionados por el INE.


In [3]:
file = r'/content/drive/MyDrive/Colab Notebooks/Python/provincias_temas_varios_2.xlsx'
datos = pd.read_excel(file, sheet_name='Datos').set_index('prov')

# Eliminamos variables dicotómicas y nos quedamos con las numéricas
cols_dicot = datos.columns.str.contains('_')
datos = datos.loc[:, ~cols_dicot]

# Limpiamos todas las observaciones que contienen valores perdidos
datos = datos.apply(pd.to_numeric, axis=1, errors='coerce').dropna()

# Visualizamos las primeras filas
datos.head()


,neag,sagt,sagu,benf,acem,acti,ocup,para,ejec,inds,cnst,ctrh,serv,pibc,ipib,inmi,pobl,espa
prov,,,,,,,,,,,,,,,,,,
Almería,22697.0,327346.0,234621.0,38299.166667,4383367.93,341.100,281.025,67.175,2026.0,2135.0,5703.0,18704.0,17550.0,19919.0,0.772923,8013.0,716820.0,570912.0
Cádiz,10069.0,540571.0,424849.0,75581.583333,5611899.98,563.325,423.125,154.950,454.0,2982.0,5916.0,27266.0,27210.0,18050.0,0.700400,6119.0,1240155.0,1194830.0
Córdoba,36641.0,1024515.0,844019.0,62022.833333,9458693.48,370.725,285.175,95.750,193.0,4196.0,5198.0,20196.0,18736.0,18525.0,0.718831,2966.0,782979.0,761876.0
Granada,41243.0,75827.0,625674.0,60393.416667,4635804.67,429.175,333.800,105.625,906.0,3494.0,7240.0,24650.0,26000.0,18181.0,0.705483,6628.0,914678.0,853709.0
Huelva,11952.0,561353.0,32997.0,41255.083333,2639859.60,249.700,194.975,58.925,269.0,1378.0,2682.0,11559.0,10136.0,20273.0,0.786659,3830.0,521870.0,477032.0


En un PCA los datos siempre deben estar centrados, es decir que la media de cada variable sea 0, lo cual se consigue restando a cada valor la media de la variable a la que corresponde. Además para evitar que las diferencias de escala de las variables generen sobrerrepresentaciones es interesante estandarizar los datos dividiéndolos por la desviación típica de las variables. De este modo las desviaciones típicas de las variables valdran 1 y la matriz de covarianzas será idéntica a la matriz de correlaciones como podremos comprobar.  


In [4]:
# Centramos los datos
datos_cent = datos.subtract(datos.mean())

# Alimentamos el estimador con los datos
datos_std = datos_cent / datos_cent.std(axis=0)

# Comprobamos que los datos están centrados y estandarizados
pd.DataFrame(datos_std, columns=datos.columns).describe().round(1).iloc[1:3, :]

,neag,sagt,sagu,benf,acem,acti,ocup,para,ejec,inds,cnst,ctrh,serv,pibc,ipib,inmi,pobl,espa
mean,0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Construcción del análisis  
El primer paso será la obtención de la matriz de varianzas y covarianzas de nuestro conjunto de datos. Esta matriz puede calcularse paso a paso de acuerdo a la siguiente expresión:
\begin{equation} 
\mathbf{S}={A}^\intercal{A} / n-1\\
\end{equation}
Siendo $A$ nuestra matriz de datos y $n$ el número de observaciones en nuestros datos.  
Por suerte esta operación está incluída en la librería `numpy`

In [18]:
# Matriz varianzas y covarianzas
S = datos_std.cov()

# visualizamos una muestras de las 3 primeras filas y columnas
pd.DataFrame(S, index=datos.columns, columns=datos.columns).iloc[:3, :3]

,neag,sagt,sagu
neag,1.000000,0.383759,0.318163
sagt,0.383759,1.000000,0.813661
sagu,0.318163,0.813661,1.000000


Podemos comprobar a primera vista que al usar datos estandarizados las matrices de covarianzas y correlaciones son idénticas

In [19]:
R = datos_std.corr()

pd.DataFrame(R, index=datos.columns, columns=datos.columns).iloc[:3, :3]

,neag,sagt,sagu
neag,1.000000,0.383759,0.318163
sagt,0.383759,1.000000,0.813661
sagu,0.318163,0.813661,1.000000


__Autovalores y autovectores de la matriz de covarianzas__

Los vectores y los valores propios de la matriz de covarianzas (o de correlaciones) se corresponden con los vectores directores de las componentes principales o factores obtenidos y las varianzas explicadas por cada componente.  
A cada autovalor ${\lambda}_i$ de la matriz $\mathbf{S}$ le corresponde un autovector $\vec{v}_i$ tal que:

\begin{equation} 
\mathbf{S}\vec{v}_i={\lambda}_i\vec{v}_i\\
\end{equation}

Podemos obtenerlo sin necesidad de resolver el sistema de ecuaciones características de la matriz de covarianzas:

In [31]:
vals_prop, vects_prop = np.linalg.eig(S)

# los vectores propios serán 
u = [col for col in vects_prop.T.tolist()]

# comprobamos visualmente que se corresponden con los obtenidos en el notebook PCA(I)
pd.DataFrame(np.transpose(vects_prop).round(3), columns=datos.columns).head(3)

,neag,sagt,sagu,benf,acem,acti,ocup,para,ejec,inds,cnst,ctrh,serv,pibc,ipib,inmi,pobl,espa
0,0.011,-0.092,-0.064,0.260,0.271,0.285,0.284,0.270,0.201,0.277,0.282,0.285,0.282,0.087,0.087,0.279,0.285,0.284
1,-0.439,-0.388,-0.348,-0.173,-0.063,-0.013,0.002,-0.123,-0.130,-0.033,0.018,-0.026,0.016,0.482,0.482,0.037,-0.022,-0.031
2,-0.030,0.521,0.605,-0.099,0.128,0.032,0.047,-0.080,-0.225,0.040,0.026,0.002,0.067,0.365,0.365,0.028,0.022,0.026


Una de las propiedades de las componentes principales es que son ortonormales, por lo tanto:  
\begin{equation} 
\vec{v}_i\vec{v}_j=0; \forall i \ne j
\end{equation}  



In [36]:
# Generamos todas las combinaciones posibles de productos i x j, i != j
indices = list(range(datos.shape[1]))
combos = itertools.combinations(indices, 2)

# vector que contiene todos los productos escalares entre vectores propios
ceros = [np.array(u[i]) @ np.array(u[j]) for i, j in combos]

#comprobación de que todos los productos son igual a 0
np.allclose(ceros, np.zeros(len(ceros)))

True

Además para cada vector $\vec{u}_i$ tenemos que $\sum_{j}{u}_{ij}^2=1$

In [37]:
# Generamos un vectos que contiene todas las sumas de cuadrados
unos = [sum(col**2) for col in vects_prop.T]

# Comprobamos que todos son igual a 11
np.allclose(unos, np.ones(len(unos)))


True